## Introduction to Data Ingestion

In [1]:
import os
from typing import List, Dict, Any
import pandas as pd


In [ ]:
from langchain_core.documents import Document
from langchain.document_loaders import(
    TextLoader,
    DirectoryLoader,
    UnstructuredFileLoader,
    PyPDFLoader,
    Docx2txtLoader,
    CSVLoader
)

### Understand the document structure

In [3]:
# Create a sample document
doc = Document(
    page_content="This is a sample document for testing purposes.",
    metadata={
        "source": "sample.txt",
        "page":1,
        "author": "Beniamine Nahid",
        "date": "2025-09-04",
        "custom_field": "any_value"

    }
)

print("Document Structure:")
print(doc)
print("\nDocument Content:")
print(doc.page_content)
print("\nDocument Metadata:")
print(doc.metadata)

Document Structure:
page_content='This is a sample document for testing purposes.' metadata={'source': 'sample.txt', 'page': 1, 'author': 'Beniamine Nahid', 'date': '2025-09-04', 'custom_field': 'any_value'}

Document Content:
This is a sample document for testing purposes.

Document Metadata:
{'source': 'sample.txt', 'page': 1, 'author': 'Beniamine Nahid', 'date': '2025-09-04', 'custom_field': 'any_value'}


### Text Files

In [1]:
# create a sample text file 
import os
os.makedirs("data/text_files", exist_ok=True)

In [2]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}



In [3]:
for filepath, content in sample_texts.items():
    with open(filepath, "w", encoding="utf-8") as f: # encodding utf-8 means it can handle all types of text files 
        f.write(content)
print("Sample text files created!")

Sample text files created!


### TextLoader

In [4]:
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

# Load a single text file
loader = TextLoader("data/text_files/python_intro.txt", encoding="utf-8")

documents = loader.load()

print(f"Number of documents loaded: {len(documents)}")
print(f"Content preview: {documents[0].page_content[:200]}...")
print(f"Metadata: {documents[0].metadata}")

Number of documents loaded: 1
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has b...
Metadata: {'source': 'data/text_files/python_intro.txt'}


### DirectoryLoader Multiple Text Files

In [7]:
from langchain_community.document_loaders import DirectoryLoader

# Load multiple text files from a directory
dir_loader = DirectoryLoader(
    "data/text_files",
    glob="**/*.txt", # Load all .txt files in the directory and subdirectories
    loader_cls=TextLoader,
    loader_kwargs= {"encoding": "utf-8"},# Specify encoding to handle various text files
    show_progress=True                            
)

documents = dir_loader.load()

for i, doc in enumerate(documents):
    print(f"\nDocument {i+1}:")
    print(f"Content preview: {doc.page_content[:200]}...")
    print(f"Metadata: {doc.metadata}")

100%|██████████| 2/2 [00:00<00:00, 368.81it/s]


Document 1:
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has b...
Metadata: {'source': 'data/text_files/python_intro.txt'}

Document 2:
Content preview: Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing...
Metadata: {'source': 'data/text_files/machine_learning.txt'}


In [8]:
# 📊 Analysis
print("\nDirectoryLoader Characteristics:")
print("✅ Advantages:")
print("  - Loads multiple files at once")
print("  - Supports glob patterns")
print("  - Progress tracking")
print("  - Recursive directory scanning")

print("\n❌ Disadvantages:")
print("  - All files must be same type")
print("  - Limited error handling per file")
print("  - Can be memory intensive for large directories")


DirectoryLoader Characteristics:
✅ Advantages:
  - Loads multiple files at once
  - Supports glob patterns
  - Progress tracking
  - Recursive directory scanning

❌ Disadvantages:
  - All files must be same type
  - Limited error handling per file
  - Can be memory intensive for large directories


### Text Splitting Strategies 

In [16]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    TokenTextSplitter,
    RecursiveCharacterTextSplitter
)

print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'), Document(metadata={'source': 'data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervised Learn

In [11]:
text = documents[0].page_content
text

'Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'

In [20]:
char_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=200,
    chunk_overlap=20,
    length_function=len 

)

char_chunks = char_splitter.split_text(text)
print(f"Number of character-based chunks: {len(char_chunks)}")
print(f"First chunk preview: {char_chunks[0]}...")

Number of character-based chunks: 3
First chunk preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has...


In [22]:
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=[" "],
    chunk_size=200,
    chunk_overlap=20,
    length_function=len 

)

recursive_chunks = recursive_splitter.split_text(text)
print(f"Number of character-based chunks: {len(recursive_chunks)}")
print(f"First chunk preview: {recursive_chunks[0]}...")

Number of character-based chunks: 3
First chunk preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has...


In [23]:
# Create text without natural break points
simple_text = "This is sentence one and it is quite long. This is sentence two and it is also quite long. This is sentence three which is even longer than the others. This is sentence four. This is sentence five. This is sentence six."

splitter = RecursiveCharacterTextSplitter(
    separators=[" "],  # Only split on spaces
    chunk_size=80,
    chunk_overlap=20,
    length_function=len
)

chunks = splitter.split_text(simple_text)

print(f"\nSimple text example - {len(chunks)} chunks:\n")

for i in range(len(chunks) - 1):
    print(f"Chunk {i+1}: '{chunks[i]}'")
    print(f"Chunk {i+2}: '{chunks[i+1]}'")
    
    
    print()


Simple text example - 4 chunks:

Chunk 1: 'This is sentence one and it is quite long. This is sentence two and it is also'
Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'

Chunk 2: 'two and it is also quite long. This is sentence three which is even longer than'
Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'

Chunk 3: 'is even longer than the others. This is sentence four. This is sentence five.'
Chunk 4: 'is sentence five. This is sentence six.'



In [25]:
token_splitter = TokenTextSplitter(
    chunk_size=50,
    chunk_overlap=10
)


token_chunks = token_splitter.split_text(text)
print(f"Number of token-based chunks: {len(token_chunks)}")
print(f"First token chunk preview: {token_chunks[0][:100]}...")

Number of token-based chunks: 3
First token chunk preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for ...


In [26]:
# 📊 Comparison
print("\n📊 Text Splitting Methods Comparison:")
print("\nCharacterTextSplitter:")
print("  ✅ Simple and predictable")
print("  ✅ Good for structured text")
print("  ❌ May break mid-sentence")
print("  Use when: Text has clear delimiters")

print("\nRecursiveCharacterTextSplitter:")
print("  ✅ Respects text structure")
print("  ✅ Tries multiple separators")
print("  ✅ Best general-purpose splitter")
print("  ❌ Slightly more complex")
print("  Use when: Default choice for most texts")

print("\nTokenTextSplitter:")
print("  ✅ Respects model token limits")
print("  ✅ More accurate for embeddings")
print("  ❌ Slower than character-based")
print("  Use when: Working with token-limited models")


📊 Text Splitting Methods Comparison:

CharacterTextSplitter:
  ✅ Simple and predictable
  ✅ Good for structured text
  ❌ May break mid-sentence
  Use when: Text has clear delimiters

RecursiveCharacterTextSplitter:
  ✅ Respects text structure
  ✅ Tries multiple separators
  ✅ Best general-purpose splitter
  ❌ Slightly more complex
  Use when: Default choice for most texts

TokenTextSplitter:
  ✅ Respects model token limits
  ✅ More accurate for embeddings
  ❌ Slower than character-based
  Use when: Working with token-limited models
